In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/AD Identification using SATD'

Mounted at /content/drive
/content/drive/My Drive/AD Identification using SATD


#Import basic libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import LabelEncoder


from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder

from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC



#Read the datasets

In [ ]:
import pandas as pd
liu_ = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
liu_ = pd.read_csv(liu_, low_memory=False)

In [ ]:
liu_['Comments'].fillna('', inplace=True)
liu_['TDType'] = liu_['TDType'].astype(str)

values_to_remove = ['MULTITHREAD', 'nan', 'removeType']
replacement_value = 'WITHOUT_CLASSIFICATION'
liu_['TDType'].replace(values_to_remove, replacement_value, inplace=True)

liu_['Comments'] = liu_['Comments'].str.replace('content=', '', regex=False)
liu_['Comments'] = liu_['Comments'].str.replace('"', '', regex=False)

In [ ]:
# Remove duplicates
liu_ = liu_.drop_duplicates(subset=['Comments', 'TDType'])


In [ ]:
# Count the number of duplicate rows in the DataFrame
num_duplicates = liu_.duplicated().sum()

print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 0


In [ ]:
liu_['TDType'] = liu_['TDType'].replace('removeType', 'WITHOUT_CLASSIFICATION')

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from scipy.sparse import csr_matrix, hstack

df = liu_
X = df['Comments'].apply(lambda x: x.lower().strip())
y = df['TDType']

# Split the dataset first to prevent leakage
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Custom feature extraction function
def custom_features(text):
    keywords = ['shape', 'input', 'tensor', 'output', 'size', 'convolution', 'value', 'efficient', 'matrix', 'model', 'node', 'function', 'batch']
    features = {f'contains_{kw}': int(kw in text.lower()) for kw in keywords}
    return pd.Series(features)

# Apply custom feature extraction separately on train and test sets
custom_features_train_df = X_train_raw.apply(custom_features)
custom_features_test_df = X_test_raw.apply(custom_features)

# Convert custom features to sparse format
custom_features_train_sparse = csr_matrix(custom_features_train_df.values)
custom_features_test_sparse = csr_matrix(custom_features_test_df.values)

vectorizer = CountVectorizer()
countVect_train = vectorizer.fit_transform(X_train_raw)
countVect_test = vectorizer.transform(X_test_raw)

X_train_combined = hstack((countVect_train, custom_features_train_sparse))
X_test_combined = hstack((countVect_test, custom_features_test_sparse))

# Train the model
model = LogisticRegression(random_state=42, solver='liblinear', penalty='l2', C=10, class_weight='balanced', max_iter=1000)
model.fit(X_train_combined, y_train)

y_pred = model.predict(X_test_combined)

print(f'F1 Score (Macro): {f1_score(y_test, y_pred, average="macro"):.2f}')
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


F1 Score (Macro): 0.68
Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.55      0.51      0.53       200
         COMPATIBILITY       0.63      0.52      0.57        89
                DEFECT       0.49      0.53      0.51       135
                DESIGN       0.87      0.83      0.85      2206
         DOCUMENTATION       0.67      0.52      0.59        23
        IMPLEMENTATION       0.70      0.74      0.72       387
                  TEST       0.73      0.77      0.75       143
WITHOUT_CLASSIFICATION       0.95      0.97      0.96      4592

              accuracy                           0.89      7775
             macro avg       0.70      0.67      0.68      7775
          weighted avg       0.89      0.89      0.89      7775

Confusion Matrix:
[[ 102    1    4   55    0    7    3   28]
 [   4   46    2   19    0    4    5    9]
 [   5    4   71   33    0    4    4   14]
 [  40   17   46 1827    3   97   16

In [ ]:
import shap

In [ ]:
explainer = shap.LinearExplainer(model, X_train_combined)
shap_values = explainer.shap_values(X_test_combined)


In [ ]:
shap.summary_plot(shap_values, X_test_combined)